In [0]:
#@title
import pandas as pd

data = {'Ob1':[1.65, 1.70, 1.40, 2.10],
        'Ob2':[1.72, 1.85, 1.75, 1.95],
        'Ob3':[1.50, 1.46, 1.38, 1.65],
        'Ob4':[1.37, 1.46, 1.38, 1.65],
        'Ob5':[1.60, 1.80, 1.55, 1.88]
        }
 
# Create DataFrame
df = pd.DataFrame(data)

def yiDot(df, i):
  return df.iloc[i].sum()

def yiDotMean(df, i):
  return yiDot(df, i) / len(df.iloc[i])

def yDotDot(df):
  return df.sum().sum()

def yDotDotMean(df):
  return yDotDot(df) / df.size

def SSError(df):
  sum = 0
  (rows, cols) = df.shape
  
  for i in range(rows):
    yiDotMean_val = yiDotMean(df, i)
    for j in range(cols):
      sum += (df.iloc[i, j] - yiDotMean_val) ** 2
  return sum

def SSTratamiento(df):
  sum = 0
  (rows, cols) = df.shape
  yDotDotMean_val = yDotDotMean(df)
  
  for i in range(rows):
    sum += (yiDotMean(df, i) - yDotDotMean_val) ** 2
  return sum * cols

def SST(df):
  return SSError(df) + SSTratamiento(df)
  
def MSTratamiento(df):
  (rows, cols) = df.shape
  return(SSTratamiento(df) / (rows - 1))

def MSError(df):
  (rows, cols) = df.shape
  return SSError(df) / (rows * (cols - 1))

def F0(df):
  return MSTratamiento(df) / MSError(df)

def printTable(table):
  col_width = max(len(word) for row in table for word in row) + 2  # padding
  for row in table:
      print("".join(word.ljust(col_width) for word in row))
  print("")

def ANOVA(df):
  (rows, cols) = df.shape

  table = [["Source of variation", "Sum of squares", "Degrees of freedom", "Mean Square", "F0"],
           ["Treatments", str(SSTratamiento(df)), str(rows - 1), str(MSTratamiento(df)), str(F0(df))],
           ["Error", str(SSError(df)), str(rows*(cols - 1)), str(MSError(df)), ""],
           ["Total", str(SST(df)), str(rows*cols - 1), "", ""]]

  printTable(table)


# Estadística para Ciencia de los Datos
# Tarea #4
## Esteban García Solís

## Parte 1

Cuatro plantas químicas, que producen los mismos productos y son propiedad de la misma compañía, 
descargan aguas residuales en arroyos en la cercanía donde se encuentran ubicadas. Para vigilar la magnitud de la contaminación creada por las aguas residuales y para determinar si esto difiere de una planta a otra, la compañía recolectó aleatoriamente  cinco muestras (observaciones) de desechos líquidos de cada planta. Cada observación tomada indica la cantidad de libras por galón de desechos. Los datos se presentan en la siguiente tabla:

|Planta | Ob. 1 | Ob. 2   | Ob. 3   | Ob. 4   | Ob. 5   |
|--- | --- |    |    |    |    | 
|**A**|1.65|1.72|1.50|1.37|1.60
|**B**|1.70|1.85|1.46|1.46|1.80
|**C**|1.40|1.75|1.38|1.38|1.55
|**D**|2.10|1.95|1.65|1.65|1.88

¿La información aporta suficiente evidencia para indicar una diferencia en el peso medio de las aguas residuales por galón descargadas de las cuatro plantas? Pruebe usando $\alpha =0.05$ (70 puntos)

R/ Se desea probar que el peso medio de las aguas residuales es igual para las plantas A, B, C y D

1.   El párametro de interés es $\mu$
2.   $H_{0}:\mu_{1}=\mu_{2}=\mu_{3}=\mu_{4}$
3.   $H_{1}: \mu_{i}\neq\mu_{j} \space \text{para al menos un par}(i,j)$
4.   Nivel de significancia $\alpha=0.05$
5.   Estadístico de prueba $F_0=\dfrac{\textrm{MS}_\textrm{tratamiento}}{\textrm{MS}_\textrm{error}}$ 
6.   Rechazamos $H_0$ en caso que: $F_0&gt;F_{\alpha,a-1,N-a}$, $F_0&gt;F_{0.95,3,16}=F_0&gt;3.23887$
7.   Cálculamos el estadístico:









In [13]:
from scipy.stats import f
print("F0.95,3,16= " + str(f.ppf(0.95,3,16)) + "\n")
ANOVA(df)

F0.95,3,16= 3.238871517453585

Source of variation   Sum of squares        Degrees of freedom    Mean Square           F0                    
Treatments            0.34860000000000047   3                     0.11620000000000015   3.9675629534784496    
Error                 0.46860000000000024   16                    0.029287500000000015                        
Total                 0.8172000000000007    19                                                                



8.   $3.9675&gt;3.23887$ Por lo tanto se rechaza $H_0$ con un nivel de significancia de 0.05, al menos una de las plantas difiere en el peso medio de desechos en las aguas residuales.

## Parte 2

Un problema recurrente en ciencia de los datos es qué hacer con datos faltantes (*missing values*). Al proceso a través del cuál solucionamos este problema se le llama *imputación de datos* o tratamiento de valores inválidos. Se recomienda cuando la cantidad de datos faltantes es poca.

- Investigue al menos tres estrategias de imputación de datos. Describa en que consiste cada una y en que casos conviene usar cada una. (30 puntos)

##Imputación usando métricas de tendencia central 
Se calcula la media, mediana o la moda usando los datos disponibles en la columna y se usa dicho valor para completar los datos faltantes.

*Ventajas:*
-	Funciona bien con conjuntos de datos numéricos pequeños (usando media o mediana)
-	Es una solución rápida de implementar 
-	Funciona para valores categóricos (usando la moda)

*Desventajas:*
-	Reduce la variabilidad de los datos
-	Puede generar un sesgo en los datos
-	No funciona en datos categóricos
-	No contempla la correlación entre características 

##Sustitución/recuperación
Tratar de recuperar el valor faltante o remplazar la muestra con una nueva que no es parte del conjunto de datos. Esta opcion es viable cuando hay acceso a la fuente de los datos.

*Ventajas:*
-	No introduce sesgos ni alteraciones al conjunto de datos.

*Desventajas:*
-	No siempre va a ser posible recuperar la información ni obtener una nueva muestra del caso en estudio.
-	Si los datos faltantes son muchos puede resultar impráctico.

##Imputación KNN
El algoritmo usa "similitud de características" para predecir los valores faltantes en el conjunto de datos. Esto significa que al dato faltante se le asigna un valor en función del parecido que tenga a otros puntos en el conjunto de entrenamiento. Para este método se define una cantidad “k” de vecinos que son seleccionados en base a una métrica de distancia. Luego los valores de estos vecinos son promediados para usar como valor estimado para la imputación

*Ventajas:*
-	Puede ser más preciso que el método de imputación usando métricas de tendencia central

*Desventajas:*
-	Su ejecución puede consumir mucho tiempo en conjuntos de datos grandes
-	Es sensible a valores extraños 

###Referencias
*Georgios Drakos, (2018, Jul 31), Handling Missing Values in Machine Learning, Retrieved 2019, Oct 14, from https://towardsdatascience.com/handling-missing-values-in-machine-learning-part-2-222154b4b58e*

*Wale Akinfaderin, (2017, Sep 11), Missing Data Conundrum: Exploration and Imputation Techniques, Retrieved 2019, Oct 14, from https://medium.com/ibm-data-science-experience/missing-data-conundrum-exploration-and-imputation-techniques-9f40abe0fd87*

*Yohan Obadia, (2017, Jan 31), The use of KNN for missing values, Retrieved 2019, Oct 14, from, https://towardsdatascience.com/the-use-of-knn-for-missing-values-cf33d935c637*

*Will Badr, (2019, Jan 5), 6 Different Ways to Compensate for Missing Values In a Dataset, Retrieved 2019, Oct 14, from https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779*

